#Entrenamiento y evaluacion del modelo

Para ejecutar el primer entrenamiento se debe cargar el dataset.csv y ejecutar el codigo, cada entrenamiento que genere mejores scores que el entrenamiento anterior se guardara en la cuenta de google drvie, en una carpeta llamada "model_saved" y para proximos entrenamientos buscara cargar un modelo desde esta carpeta si existe

Variables importantes:
num_epochs = 10
**#determina la cantidad de epocas que se va a realizar el entrenamiento**
if avg_loss <= 0.04: **#en caso de que antes de terminar la cantidad de epocas definida se alcance un avg_loss deseado, se puede parar el entrenamiento**


#instalaciones previas

para la generacion de las metricas de calidad

In [ ]:
pip install rouge

In [ ]:
import pandas as pd
import os
import random
from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from rouge import Rouge
from sklearn.metrics import accuracy_score
import re

class CodeTranslationDataset(Dataset):
    def __init__(self, data_pairs, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Pre-procesamiento de todos los datos de una vez
        self.inputs = []
        self.targets = []

        for input, target in data_pairs:
            # Limpieza básica
            input = input.strip('"')
            target = target.strip('"')
            # target = target.replace('\\n', '\n').replace('\\t', '\t')

            self.inputs.append(input)
            self.targets.append(target)

        # Tokenizar todos los ejemplos de una vez (más eficiente)
        self.tokenized_inputs = self.tokenizer(
            self.inputs,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        self.tokenized_targets = self.tokenizer(
            self.targets,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_inputs['input_ids'][idx],
            'attention_mask': self.tokenized_inputs['attention_mask'][idx],
            'labels': self.tokenized_targets['input_ids'][idx]
        }

def load_dataset(file_path):
    """Carga el dataset usando pandas directamente"""
    try:
        # Primero intentamos con UTF-8, la codificación más común
        df = pd.read_csv(file_path, header=None, encoding='utf-8')
    except UnicodeDecodeError:
        # Si falla, probamos con latin1 que es más permisiva
        df = pd.read_csv(file_path, header=None, encoding='latin1')

    dataset_pairs = []
    for _, row in df.iterrows():
        if len(row) >= 2:
            input = str(row[0]).strip().strip('"')
            target = str(row[1]).strip().strip('"')
            dataset_pairs.append((input, target))

    print(f"Se cargaron {len(dataset_pairs)} ejemplos del dataset")
    return dataset_pairs

def load_or_train_model(model_dir='/content/drive/MyDrive/Colab Notebooks/best_model'):
    # Verificar si existe un modelo guardado
    if os.path.exists(model_dir):
        print(f"Cargando modelo guardado desde {model_dir}")
        tokenizer = T5Tokenizer.from_pretrained(model_dir)
        model = T5ForConditionalGeneration.from_pretrained(model_dir)

        # Cargar mejor pérdida previa
        loss_history_file = os.path.join(model_dir, 'loss_history.txt')
        if os.path.exists(loss_history_file):
            with open(loss_history_file, 'r') as f:
                best_loss = float(f.read().strip())
            print(f"Cargando mejor pérdida previa: {best_loss:.4f}")
            return model, tokenizer, best_loss > 0.04  # Solo entrenar si la pérdida es mayor que 0.04
    else:
        print("No se encontró modelo guardado. Iniciando con t5-small base")
        tokenizer = T5Tokenizer.from_pretrained('t5-small')
        model = T5ForConditionalGeneration.from_pretrained('t5-small')

    # Si llegamos aquí, se requiere entrenamiento
    return model, tokenizer, True

def train_model(model, tokenizer, dataset_pairs, model_dir='/content/drive/MyDrive/Colab Notebooks/best_model'):
    # Mover modelo a GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Crear dataset y dataloader con batch_size optimizado
    batch_size = 16 if torch.cuda.is_available() else 8
    dataset = CodeTranslationDataset(dataset_pairs, tokenizer, max_length=128)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Configurar optimizador con un learning rate apropiado
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

    # Scheduler más adecuado para este tipo de tarea
    num_epochs = 10
    total_steps = len(dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # Cargar mejor pérdida previa si existe
    best_loss = float('inf')
    os.makedirs(model_dir, exist_ok=True)
    loss_history_file = os.path.join(model_dir, 'loss_history.txt')
    if os.path.exists(loss_history_file):
        with open(loss_history_file, 'r') as f:
            best_loss = float(f.read().strip())
        print(f"Cargando mejor pérdida previa: {best_loss:.4f}")

    # Variable para guardar solo cada 2 épocas o cuando haya mejora significativa
    save_epoch_interval = 2
    min_improvement = 0.01  # 1% de mejora para guardar el modelo

    # Entrenamiento
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        model.train()

        # Usar tqdm para mostrar progreso si está disponible
        try:
            from tqdm import tqdm
            iterator = tqdm(dataloader, desc=f"Época {epoch+1}/{num_epochs}")
        except ImportError:
            iterator = dataloader

        for batch in iterator:
            optimizer.zero_grad()

            # Mover batch a GPU si está disponible
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Habilitar mixed precision si está disponible
            with torch.cuda.amp.autocast() if torch.cuda.is_available() else contextlib.nullcontext():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()

            # Gradient clipping para evitar explosión de gradientes
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            scheduler.step()

        avg_loss = total_loss/len(dataloader)
        print(f"Época {epoch+1}/{num_epochs}, Loss promedio: {avg_loss:.4f}")

        # Guardar el modelo solo si hay una mejora significativa o cada save_epoch_interval
        should_save = False

        if avg_loss < best_loss - min_improvement:
            print(f"Mejora significativa: {best_loss:.4f} -> {avg_loss:.4f}")
            should_save = True
        elif (epoch + 1) % save_epoch_interval == 0:
            print(f"Guardando por intervalo de época")
            should_save = True

        if should_save:
            best_loss = min(best_loss, avg_loss)
            # Guardar en el directorio especificado
            model.save_pretrained(model_dir)
            tokenizer.save_pretrained(model_dir)

            # Guardar la mejor pérdida
            with open(loss_history_file, 'w') as f:
                f.write(str(best_loss))

            print(f"Modelo guardado con loss: {avg_loss:.4f}")

        # Si alcanzamos una pérdida <= 0.04, terminamos el entrenamiento temprano
        if avg_loss <= 0.04:
            print("Se alcanzó la pérdida objetivo de 0.04 o menos. Terminando entrenamiento.")
            break

    return model, tokenizer

def translate_code(model, tokenizer, input_code, device):
    # Clean input code
    input_code = input_code.strip()

    # Tokenizar y mover a GPU de una vez
    inputs = tokenizer(input_code, return_tensors="pt", max_length=128, truncation=True).to(device)

    # Configuración de generación más eficiente
    outputs = model.generate(
        inputs.input_ids,
        max_length=128,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2,
        temperature=0.7
    )

    # Decodificar resultado
    translated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated_code = translated_code.replace('\\n', '\n').replace('\\t', '\t')

    return translated_code

def format_output_with_random_number(translated_code):
    if "Algoritmo conversion" in translated_code and not "conversion_" in translated_code:
        random_num = random.randint(1000, 9999)
        translated_code = translated_code.replace("Algoritmo conversion", f"Algoritmo conversion_{random_num}")
    return translated_code

def tokenize_code(code):
    """Tokeniza el código para evaluación de métricas de manera más adecuada para código fuente"""
    # Reemplazar espacios en blanco alrededor de símbolos comunes para que sean tokens separados
    for symbol in '(){}[]<>;:,.+-*/=&|^~!%':
        code = code.replace(symbol, f' {symbol} ')

    # Dividir por espacios en blanco y eliminar tokens vacíos
    tokens = [token for token in re.split(r'\s+', code) if token]
    return tokens

def evaluate_model_metrics(model, tokenizer, test_data, device):
    """Evalúa el modelo utilizando varias métricas comunes en NLP y específicas para código"""
    model.eval()

    print("\n=== Evaluación de Métricas del Modelo ===")

    # Preparar listas para almacenar resultados
    references_bleu = []
    hypotheses_bleu = []
    references_raw = []
    hypotheses_raw = []
    exact_match_count = 0

    # Inicializar ROUGE para evaluación
    try:
        rouge = Rouge()
        rouge_available = True
    except:
        print("ROUGE no está disponible, instalarlo con: pip install rouge")
        rouge_available = False

    # Evaluar cada par en el conjunto de prueba
    for input_code, reference_code in test_data:
        # Traducir el código de entrada
        translated_code = translate_code(model, tokenizer, input_code, device)

        # Formatear si es necesario
        translated_code = format_output_with_random_number(translated_code)

        # Tokenizar para BLEU
        reference_tokens = [tokenize_code(reference_code)]
        hypothesis_tokens = tokenize_code(translated_code)

        references_bleu.append(reference_tokens)
        hypotheses_bleu.append(hypothesis_tokens)

        # Almacenar versiones sin procesar para otras métricas
        references_raw.append(reference_code)
        hypotheses_raw.append(translated_code)

        # Comprobar coincidencia exacta
        if translated_code.strip() == reference_code.strip():
            exact_match_count += 1

    # Calcular métricas
    # 1. BLEU Score
    bleu_score = corpus_bleu(references_bleu, hypotheses_bleu)

    # 2. Exact Match (precisión exacta)
    exact_match_accuracy = exact_match_count / len(test_data) * 100

    # 3. ROUGE (si está disponible)
    if rouge_available and len(references_raw) > 0:
        try:
            rouge_scores = rouge.get_scores(hypotheses_raw, references_raw, avg=True)
            rouge_l_f = rouge_scores['rouge-l']['f'] * 100
        except Exception as e:
            print(f"Error al calcular ROUGE: {e}")
            rouge_l_f = 0
    else:
        rouge_l_f = 0

    # 4. Codebleu - aproximación simplificada
    # Aquí implementamos una métrica específica para código que evalúa palabras clave importantes
    # como "Algoritmo", "Proceso", etc.

    def calculate_keyword_match_score(refs, hyps, keywords):
        """Calcula el puntaje de coincidencia de palabras clave importantes en código"""
        total_score = 0
        for ref, hyp in zip(refs, hyps):
            ref_contains = sum(1 for kw in keywords if kw.lower() in ref.lower())
            hyp_contains = sum(1 for kw in keywords if kw.lower() in hyp.lower())

            if ref_contains > 0:
                score = hyp_contains / ref_contains
                total_score += min(score, 1.0)
            else:
                total_score += 1.0 if hyp_contains == 0 else 0.0

        return (total_score / len(refs)) * 100 if refs else 0

    # Palabras clave importantes en el contexto de tu traducción de código
    code_keywords = ["Algoritmo", "Proceso", "FinAlgoritmo", "Escribir", "Leer"]
    keyword_score = calculate_keyword_match_score(references_raw, hypotheses_raw, code_keywords)

    # 5. Estructura sintáctica - aproximación simple
    def calculate_structure_score(refs, hyps):
        """Evalúa la similitud estructural basada en indentación y símbolos de estructura"""
        structural_symbols = ["Algoritmo", "Proceso", "FinAlgoritmo", "Escribir", "Leer"]
        return calculate_keyword_match_score(refs, hyps, structural_symbols)

    structure_score = calculate_structure_score(references_raw, hypotheses_raw)

    # Imprimir resultados de métricas
    print("\n=== Resultados de Evaluación ===")
    print(f"1. BLEU Score: {bleu_score:.4f}")
    print(f"2. Exact Match Accuracy: {exact_match_accuracy:.2f}%")
    print(f"3. ROUGE-L F1: {rouge_l_f:.2f}%")
    print(f"4. Keyword Match Score: {keyword_score:.2f}%")
    print(f"5. Structure Similarity Score: {structure_score:.2f}%")

    # Calcular score compuesto ponderado
    composite_score = (
        0.3 * bleu_score * 100 +  # Ponderación de BLEU
        0.2 * exact_match_accuracy +  # Ponderación de Exact Match
        0.2 * rouge_l_f +  # Ponderación de ROUGE-L
        0.15 * keyword_score +  # Ponderación de Keyword Match
        0.15 * structure_score  # Ponderación de Structure Similarity
    )

    print(f"\nComposite Quality Score: {composite_score:.2f}%")

    # Guardar resultados en un archivo
    metrics_results = {
        'bleu': bleu_score,
        'exact_match': exact_match_accuracy / 100,
        'rouge_l': rouge_l_f / 100,
        'keyword_score': keyword_score / 100,
        'structure_score': structure_score / 100,
        'composite_score': composite_score / 100
    }

    return metrics_results

def create_test_dataset(file_path=None, sample_size=50):
    """Crea un conjunto de prueba separado para evaluación"""
    if file_path and os.path.exists(file_path):
        # Cargar desde archivo si existe
        all_data = load_dataset(file_path)

        # Asegurar que tenemos suficientes ejemplos
        if len(all_data) < sample_size:
            print(f"Advertencia: El dataset solo contiene {len(all_data)} ejemplos, se usarán todos.")
            return all_data

        # Tomar una muestra aleatoria para test
        test_indices = random.sample(range(len(all_data)), sample_size)
        return [all_data[i] for i in test_indices]
    else:
        # Datos de ejemplo para pruebas si no hay archivo
        return [
            ("1 1 1 0 0 4 1 6 'tara' 1", "Algoritmo conversion\nProceso\n    Escribir tara\nFinProceso"),
            ("1 1 1 0 0 4 1 8 'comand' 1", "Algoritmo conversion\nProceso\n    Escribir comand\nFinProceso"),
            ("1 1 1 0 0 5 1 10 'variable' 1", "Algoritmo conversion\nProceso\n    Escribir variable\nFinProceso"),
            ("1 1 1 0 0 5 1 7 'count' 1", "Algoritmo conversion\nProceso\n    Escribir count\nFinProceso"),
            # Agregar más ejemplos aquí...
        ]

if __name__ == "__main__":
    # Configurar para usar GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando dispositivo: {device}")

    # Importación condicional para mixed precision
    import contextlib
    if torch.cuda.is_available():
        print("CUDA disponible, habilitando optimizaciones adicionales")

    # Cargar o inicializar modelo
    model, tokenizer, needs_training = load_or_train_model()
    model = model.to(device)

    # Solo entrenar si es necesario
    if needs_training:
        print("Cargando dataset para entrenamiento...")
        dataset_pairs = load_dataset('dataset.csv')

        print("Iniciando entrenamiento...")
        model, tokenizer = train_model(model, tokenizer, dataset_pairs)
    else:
        print("Omitiendo entrenamiento debido a que la pérdida ya es óptima")

    # Crear conjunto de test (independiente del conjunto de entrenamiento)
    print("\nPreparando conjunto de test para evaluación...")
    test_dataset = create_test_dataset('dataset.csv', sample_size=50)
    print(f"Se utilizarán {len(test_dataset)} ejemplos para evaluación")

    # Evaluar el modelo con métricas
    print("\nEvaluando métricas del modelo...")
    metrics = evaluate_model_metrics(model, tokenizer, test_dataset, device)

    # Guardar métricas en un archivo
    metrics_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/best_model', 'metrics.txt')
    with open(metrics_file, 'w') as f:
        for metric_name, metric_value in metrics.items():
            f.write(f"{metric_name}: {metric_value:.6f}\n")
    print(f"Métricas guardadas en {metrics_file}")

    # Probar con ejemplos específicos
    test_codes = [
        "1 1 1 0 0 4 1 6 'tara' 1",
        "1 1 1 0 0 4 1 8 'comand' 1",
        "1 1 1 0 0 5 1 10 'variable' 1",
        "1 1 1 0 0 5 1 7 'count' 1",
    ]

    print("\nPruebas de traducción con ejemplos específicos:")
    for code in test_codes:
        result = translate_code(model, tokenizer, code, device)
        formatted_result = format_output_with_random_number(result)
        print(f"\nEntrada: {code}")
        print(f"Salida:\n{formatted_result}")

Usando dispositivo: cpu
Cargando modelo guardado desde /content/drive/MyDrive/Colab Notebooks/best_model
Cargando mejor pérdida previa: 0.0227
Omitiendo entrenamiento debido a que la pérdida ya es óptima

Preparando conjunto de test para evaluación...
Se utilizarán 4 ejemplos para evaluación

Evaluando métricas del modelo...

=== Evaluación de Métricas del Modelo ===


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3


=== Resultados de Evaluación ===
1. BLEU Score: 0.0000
2. Exact Match Accuracy: 0.00%
3. ROUGE-L F1: 36.36%
4. Keyword Match Score: 66.67%
5. Structure Similarity Score: 66.67%

Composite Quality Score: 27.27%
Métricas guardadas en /content/drive/MyDrive/Colab Notebooks/best_model/metrics.txt

Pruebas de traducción con ejemplos específicos:

Entrada: 1 1 1 0 0 4 1 6 'tara' 1
Salida:
Algoritmo conversion_1038 Escribir 'tara' FinAlgoritoma

Entrada: 1 1 1 0 0 4 1 8 'comand' 1
Salida:
Algoritmo conversion_8682 Escribir 'comand' FinAlgoritomic

Entrada: 1 1 1 0 0 5 1 10 'variable' 1
Salida:
Algoritmo conversion_4489 Escribir 'variable' FinAlgoritomic

Entrada: 1 1 1 0 0 5 1 7 'count' 1
Salida:
Algoritmo conversion_2509 Escribir 'count' FinAlgoritoMo


#Creacion de la API con flask

Con este codigo se levantara un servidor web en flask, con un endpoint "/translate" para hacer un un post de un archivo .dfd, y hacer uso del modelo previamente entrenado para la traduccion, por defecto corre en el puerto 5000, se generan dos accesos: uno local para acceder dentro de el entorno de google colab, y un tunel web para acceder desde otra maquina, el enlace del tunel se puede encontrar en la linea anterior a **"Puedes acceder a la API usando el enlace superior o enviar solicitudes POST a /translate**
#**importante: antes de ejecutar el servidor en flask asegurarse de haber cargado el template "index.html" en los archivos de google colab**


instalar cors para evitar posibles errores de acceso

In [ ]:
!pip install flask-cors

In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from werkzeug.utils import secure_filename
import os
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import tempfile
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": ["*", "null"]}}, supports_credentials=True)
@app.after_request
def after_request(response):
    # Permitir explícitamente el origen "null" para archivos HTML locales
    origin = request.headers.get('Origin')
    if origin == 'null' or origin is None:
        response.headers.add('Access-Control-Allow-Origin', 'null')
    else:
        response.headers.add('Access-Control-Allow-Origin', origin)

    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
    response.headers.add('Access-Control-Allow-Credentials', 'true')
    return response

# Configuración básica
UPLOAD_FOLDER = '/content/uploads'
OUTPUT_FOLDER = '/content/outputs'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Cargar modelo y tokenizador (esto se ejecutará solo una vez al iniciar la API)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

model = None
tokenizer = None

def load_model():
    global model, tokenizer
    model_dir = '/content/drive/MyDrive/Colab Notebooks/best_model'
    print(f"Cargando modelo desde {model_dir}")
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    model = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
    print("Modelo cargado exitosamente")

load_model()

def process_dfd_file(file_path):
    """Procesa el archivo .dfd según los requisitos"""
    try:
        # Leer el archivo .dfd como texto
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Eliminar la primera línea
        if len(lines) > 0:
            lines = lines[1:]

        # Unir todas las líneas en una sola, eliminando saltos de línea y espacios extras
        single_line = ' '.join(line.strip() for line in lines)

        return single_line
    except Exception as e:
        print(f"Error procesando archivo: {e}")
        return None

def translate_code(input_code):
    """Traduce el código usando el modelo cargado"""
    try:
        # Limpiar el código de entrada
        input_code = input_code.strip()

        # Tokenizar y mover a GPU
        inputs = tokenizer(input_code, return_tensors="pt", max_length=128, truncation=True).to(device)

        # Generar la traducción
        outputs = model.generate(
            inputs.input_ids,
            max_length=128,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=2,
            temperature=0.7
        )

        # Decodificar el resultado
        translated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translated_code = translated_code.replace('\\n', '\n').replace('\\t', '\t')

        return translated_code
    except Exception as e:
        print(f"Error en la traducción: {e}")
        return None

@app.route('/translate', methods=['POST'])
def translate_file():
    """Endpoint principal para la traducción de archivos"""
    if 'file' not in request.files:
        return jsonify({'error': 'No se proporcionó archivo'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'Nombre de archivo vacío'}), 400

    # Verificar que el archivo tenga extensión .dfd
    if not file.filename.lower().endswith('.dfd'):
        return jsonify({'error': 'El archivo debe tener extensión .dfd'}), 400

    try:
        # Guardar el archivo temporalmente
        filename = secure_filename(file.filename)
        input_path = os.path.join(UPLOAD_FOLDER, filename)
        file.save(input_path)

        # Procesar el archivo .dfd
        processed_code = process_dfd_file(input_path)
        if processed_code is None:
            return jsonify({'error': 'Error al procesar el archivo .dfd'}), 500

        # Traducir el código
        translated_code = translate_code(processed_code)
        if translated_code is None:
            return jsonify({'error': 'Error al traducir el código'}), 500

        # Crear archivo de salida .txt
        output_filename = os.path.splitext(filename)[0] + '_translated.txt'
        output_path = os.path.join(OUTPUT_FOLDER, output_filename)

        with open(output_path, 'w') as f:
            f.write(translated_code)

        # Enviar el archivo de vuelta al cliente
        return send_file(
            output_path,
            as_attachment=True,
            download_name=output_filename,
            mimetype='text/plain'
        )
    except Exception as e:
        return jsonify({'error': f'Error en el servidor: {str(e)}'}), 500
    finally:
        # Limpiar archivos temporales
        if 'input_path' in locals() and os.path.exists(input_path):
            os.remove(input_path)
        # No eliminamos el output para que el cliente pueda descargarlo

@app.route('/test', methods=['GET'])
def test_api():
    """Endpoint de prueba para verificar que la API está funcionando"""
    return jsonify({
        'status': 'API funcionando',
        'model_loaded': model is not None,
        'device': str(device)
    })

def run_in_colab():
    """Función para ejecutar la API en Google Colab"""
    from google.colab import output
    import threading

    # Iniciar Flask en un hilo separado
    threading.Thread(target=app.run, kwargs={
        'host': '0.0.0.0',
        'port': 5000
    }).start()

    # Configurar el proxy en Colab
    print("La API se está ejecutando en:")
    output.serve_kernel_port_as_window(5000)
    print("Puedes acceder a la API usando el enlace superior o enviar solicitudes POST a /translate")

if __name__ == '__main__':
    # Si se ejecuta en Colab, usar run_in_colab()
    # Si se ejecuta localmente, usar app.run()
    try:
        from google.colab import drive
        # Montar Google Drive si es necesario
        drive.mount('/content/drive')
        run_in_colab()
    except:
        app.run(host='0.0.0.0', port=5000, debug=True)

Usando dispositivo: cpu
Cargando modelo desde /content/drive/MyDrive/Colab Notebooks/best_model
Modelo cargado exitosamente
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
La API se está ejecutando en:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Puedes acceder a la API usando el enlace superior o enviar solicitudes POST a /translate
